In [1]:
from IPython.display import display 
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image

from sklearn.linear_model import RidgeClassifierCV
from sklearn.metrics import confusion_matrix, balanced_accuracy_score
from sklearn.preprocessing import StandardScaler
from sktime.transformations.panel.rocket import (MiniRocket, MultiRocket)

from utils import *

In [2]:
import sktime

In [3]:
df = flattened_array_ordering(dimensions =(28,28), verbose = True)

In [4]:
# Create a DataFrame with different pixel orderings 
# by position in a flattened row-wise array

df = convert_to_numeric_string(df)

df.head(5)

,row_wise,col_wise,diagonal_1,diagonal_2
0,00,00,756,00
1,01,28,728,28
2,02,56,757,01
3,03,84,700,56
4,04,112,729,29


In [5]:
# load the MNIST train/test dataset from .csv files

train = pd.read_csv('./MNIST_CSV/mnist_train.csv', 
                    names=['label']+list(df['row_wise']), 
                    dtype='uint8')
test = pd.read_csv('./MNIST_CSV/mnist_test.csv', 
                   names=['label']+list(df['row_wise']), 
                   dtype='uint8')

display(train.head(5))
display(test.head(5))

,label,00,01,02,03,04,05,06,07,08,...,774,775,776,777,778,779,780,781,782,783
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


,label,00,01,02,03,04,05,06,07,08,...,774,775,776,777,778,779,780,781,782,783
0,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Append the various flattened pixel configurations to df

train = train.join(train.loc[:,df['col_wise']], 
                   rsuffix='_colwise')
test = test.join(test.loc[:,df['col_wise']], 
                 rsuffix='_colwise')

train = train.join(train.loc[:,df['diagonal_1']], 
                   rsuffix='_diagonal_1')
test = test.join(test.loc[:,df['diagonal_1']], 
                 rsuffix='_diagonal_1')

train = train.join(train.loc[:,df['diagonal_2']], 
                   rsuffix='_diagonal_2')
test = test.join(test.loc[:,df['diagonal_2']], 
                 rsuffix='_diagonal_2')

# create X and y train/test DataFrames

X_train = train.iloc[:,1:]
y_train = train.iloc[:,0]
X_test = test.iloc[:,1:]
y_test = test.iloc[:,0]

In [7]:
# Nest the df so that each row of pixel configurations 
# is represented as a single pd.Series

pd_list_train_X = pd.DataFrame([pd.Series([X_train.iloc[idx]]) for idx in range(len(X_train))])
pd_list_test_X = pd.DataFrame([pd.Series([X_test.iloc[idx]]) for idx in range(len(X_test))])

In [8]:
display(pd_list_train_X.head(5))
display(pd_list_test_X.head(5))

,0
0,00 0 01 0 02 ...
1,00 0 01 0 02 ...
2,00 0 01 0 02 ...
3,00 0 01 0 02 ...
4,00 0 01 0 02 ...


,0
0,00 0 01 0 02 ...
1,00 0 01 0 02 ...
2,00 0 01 0 02 ...
3,00 0 01 0 02 ...
4,00 0 01 0 02 ...


In [9]:
#Show the first image in the train dataset

img = Image.fromarray(pd_list_train_X.iloc[1][0][:784].values.reshape((28,28)), 'L')

display(img)

In [10]:
%%time

# n_jobs=-1 for max CPU cores and set kernels to any number
multirocket = MultiRocket(n_jobs=-1, num_kernels=2600, random_state=13)  


CPU times: user 79.8 ms, sys: 61.1 ms, total: 141 ms
Wall time: 206 ms


In [11]:
%%time

X_train_transform = multirocket.fit_transform(pd_list_train_X)
# check that shape of multirocket output is ~8x number of kernels
print("Shape of output:",X_train_transform.shape)

Shape of output: (60000, 20160)
CPU times: user 1h 22min 50s, sys: 8.25 s, total: 1h 22min 59s
Wall time: 1min 10s


In [12]:
%%time

X_test_transform = multirocket.transform(pd_list_test_X)

CPU times: user 13min 47s, sys: 26.6 s, total: 14min 14s
Wall time: 11.4 s


In [13]:
scaler = StandardScaler(with_mean=False)
classifier = RidgeClassifierCV()

In [14]:
%%time

X_train_scaled_transform = scaler.fit_transform(X_train_transform)

CPU times: user 9.91 s, sys: 2.59 s, total: 12.5 s
Wall time: 12.5 s


In [15]:
%%time

classifier.fit(X_train_scaled_transform, y_train)

CPU times: user 18h 20min 29s, sys: 4h 28min 27s, total: 22h 48min 56s
Wall time: 22min 41s


RidgeClassifierCV()

In [16]:
%%time

X_test_scaled_transform = scaler.transform(X_test_transform)

CPU times: user 1.54 s, sys: 2.93 s, total: 4.47 s
Wall time: 888 ms


In [17]:
%%time

classifier.score(X_test_scaled_transform, y_test)

CPU times: user 4.46 s, sys: 2.4 s, total: 6.86 s
Wall time: 1.03 s


0.9882

In [18]:
%%time

preds = classifier.predict(X_test_scaled_transform)

CPU times: user 4.9 s, sys: 7.16 s, total: 12.1 s
Wall time: 1.08 s


In [19]:
balanced_accuracy_score(y_test, preds)

0.9881412750611087

In [20]:
confusion_matrix(y_test, preds)

array([[ 977,    0,    0,    0,    0,    0,    1,    1,    1,    0],
       [   0, 1129,    2,    0,    1,    0,    2,    0,    0,    1],
       [   2,    0, 1022,    0,    0,    0,    0,    6,    2,    0],
       [   0,    0,    2,  999,    0,    5,    0,    3,    1,    0],
       [   1,    0,    0,    0,  969,    0,    5,    0,    0,    7],
       [   1,    0,    0,    6,    0,  881,    3,    1,    0,    0],
       [   4,    3,    1,    0,    1,    3,  945,    0,    1,    0],
       [   0,    3,    7,    3,    1,    0,    0, 1011,    0,    3],
       [   1,    0,    3,    0,    1,    0,    0,    2,  966,    1],
       [   2,    2,    2,    4,    6,    3,    0,    5,    2,  983]])